In [1]:
import pandas as pd
import numpy as np
import os

## 1 - Load the preprocessed data

In [2]:
processed_file_path = os.path.join('..', 'data', 'processed')
clustering_model_name = 'KMeans'
processed_file_name = f'train_{clustering_model_name}.csv'

train_df = pd.read_csv(os.path.join(processed_file_path, processed_file_name), low_memory=False)
train_df.head()

,file,VMONTH,VYEAR,VDAYR,YEAR,AGE,SEX,ETHNIC,RACE,USETOBAC,...,OTHPROV,MHP,NODISP,REFOTHMD,RETAPPT,OTHDISP,ERADMHOS,cluster,CombinedText,ProcessedText
0,opd2006.csv,December,2006.0,Friday,2006.0,55.0,Male,Not Hispanic or Latino,White Only,Not current,...,No,NaN,One or more dispositions marked,No,No,No,No,9,"55_year_old Male Acute problem Injury, other a...",55_year_old male acute problem injury unspecif...
1,opd2006.csv,November,2006.0,Thursday,2006.0,66.0,Male,Not Hispanic or Latino,White Only,Not current,...,No,NaN,One or more dispositions marked,No,No,No,No,0,66_year_old Male Acute problem Cough Chronic o...,66_year_old male acute problem cough chronic o...
2,opd2006.csv,November,2006.0,Wednesday,2006.0,71.0,Female,Not Hispanic or Latino,White Only,Not current,...,No,NaN,One or more dispositions marked,Yes,No,No,No,7,71_year_old Female Acute problem General ill f...,71_year_old female acute problem general ill f...
3,opd2006.csv,November,2006.0,Tuesday,2006.0,1.0,Female,Not Hispanic or Latino,White Only,Not current,...,No,NaN,One or more dispositions marked,No,No,No,No,2,1_year_old Female Acute problem Other and unsp...,1_year_old female acute problem unspecified sy...
4,opd2006.csv,November,2006.0,Monday,2006.0,21.0,Female,Not Hispanic or Latino,White Only,Current,...,No,NaN,One or more dispositions marked,No,No,No,No,6,21_year_old Female TOBACCO user Acute problem ...,21_year_old female tobacco user acute problem ...


## 2 - Topic modeling

In [3]:
import gensim
from gensim import corpora

random_seed = 42

In [4]:
# Generate bigrams
def generate_bigrams(row):
    tokens = row.split()
    bigram_model = gensim.models.Phrases(tokens, min_count=5, threshold=100)
    tokens = [token for token in bigram_model[tokens]]
    return tokens


print(generate_bigrams(train_df['ProcessedText'].iloc[0]))
preprocessed_docs = train_df['ProcessedText'].apply(generate_bigrams)

['55_year_old', 'male', 'acute', 'problem', 'injury', 'unspecified', 'sho', 'diabetes', 'hyperlipidemia', 'obesity', 'currently', 'enrol', 'disease', 'management', 'program', 'shoulder', 'upper', 'arm', 'injury']


In [5]:
# Build dictionary and corpus
dictionary = corpora.Dictionary(preprocessed_docs)
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_docs]

In [6]:
# Train LDA model
n_topics = 10

lda_model = gensim.models.LdaMulticore(
    corpus=corpus,
    id2word=dictionary,
    num_topics=n_topics,
    random_state=random_seed,
    chunksize=100,
    passes=10,
)



### 2.1 - Visualize the topics

In [7]:
import pyLDAvis.gensim
import pyLDAvis

pyLDAvis.enable_notebook()

In [8]:
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.253345  0.010712       1        1  20.333820
0      0.129566 -0.290440       2        1  12.584457
8      0.118295  0.138833       3        1  11.458144
2     -0.220767 -0.010410       4        1   9.564243
4      0.139430  0.134646       5        1   9.540921
1      0.111756  0.124994       6        1   9.324127
9      0.038309 -0.144507       7        1   8.063826
6      0.051952 -0.037955       8        1   7.399013
3      0.041082  0.040398       9        1   7.316728
5     -0.156278  0.033727      10        1   4.414721, topic_info=            Term          Freq         Total Category  logprob  loglift
124         care  17734.000000  17734.000000  Default  30.0000  30.0000
127   preventive  17651.000000  17651.000000  Default  29.0000  29.0000
72         visit  12266.000000  12266.000000  Default  28.0000  28.0000
5        disease  32300.000000  32300.000000  Default  27.0000  27.0000
1          acute  34143.000000  34143.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
34        female   1251.828258  55649.343295  Topic10  -3.9427  -0.6742
125  examination    818.304247  20797.574569  Topic10  -4.3679  -0.1151
251   7_year_old    355.579511    991.791549  Topic10  -5.2014   2.0945
43    1_year_old    443.166898   6381.233993  Topic10  -4.9812   0.4531
92         flare    391.370918   6093.870233  Topic10  -5.1055   0.3748

[651 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
875       3  0.996786          10
43        2  0.348835  1_year_old
43        3  0.385662  1_year_old
43        5  0.023663  1_year_old
43        6  0.022723  1_year_old
...     ...       ...         ...
817       7  0.021715       wound
339       1  0.001859       wrist
339       2  0.001859       wrist
339       5  0.992753       wrist
339       7  0.001859       wrist

[1394 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 1, 9, 3, 5, 2, 10, 7, 4, 6])

## 3 - Get the topic distribution for each cluster

In [9]:
# Get the topic distribution for each document 
def get_avg_topic_distribution(cluster_data, topic_distribution, num_topics):
    cluster_topics = np.zeros((len(cluster_data.index), num_topics))

    # Update the distribution with the actual values
    for i, doc_index in enumerate(cluster_data.index.tolist()):
        for topic, prob in topic_distribution[doc_index]:
            cluster_topics[i, topic] = prob

    avg_topic_dist = np.mean(cluster_topics, axis=0)
    return avg_topic_dist


topic_distribution = lda_model.get_document_topics(corpus, minimum_probability=0.0)
clusters = train_df.groupby('cluster').ProcessedText

cluster_topics = []
for cluster, data in clusters:
    avg_topic_dist = get_avg_topic_distribution(data, topic_distribution, lda_model.num_topics)
    cluster_topics.append([avg_topic_dist.tolist()])

cluster_topics_df = pd.DataFrame(cluster_topics, columns=['avg_topic_distribution'])

cluster_topics_df

,avg_topic_distribution
0,"[0.04261925155484891, 0.08854286106026645, 0.0..."
1,"[0.5714818877344372, 0.03401331077402603, 0.04..."
2,"[0.2339972153485072, 0.04280077148575792, 0.02..."
3,"[0.05030263489993748, 0.03193899562216563, 0.2..."
4,"[0.05861542928678932, 0.11581865783028757, 0.0..."
5,"[0.08375966199840071, 0.12638677027213208, 0.1..."
6,"[0.08979103710783133, 0.07761657948772559, 0.0..."
7,"[0.04834084913826965, 0.09551950935885142, 0.0..."
8,"[0.2734567208326669, 0.040984406242198235, 0.0..."
9,"[0.10041553686998106, 0.0775235009207313, 0.10..."


### 3.2 - Visualize the distribution of topics within each cluster

In [10]:
import altair as alt

In [11]:
# Reshape the dataframe to have a row for each cluster and topic
cluster_topics_heatmap_df = cluster_topics_df.avg_topic_distribution.apply(pd.Series).reset_index().rename(columns={'index': 'cluster'}).melt(id_vars='cluster', var_name='topic', value_name='probability')

heatmap = alt.Chart(cluster_topics_heatmap_df).mark_rect().encode(
    x='topic:O',
    y='cluster:O',
    color='probability:Q'
).properties(
    title='Average Topic Distribution for Each Cluster',
    width=400,
    height=400
)

heatmap

alt.Chart(...)